In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from pprint import pprint
import gc
import os
import random

import copy
from glob import glob
import cv2
from PIL import Image
import random
from collections import deque, defaultdict
from multiprocessing import Pool, Process
from functools import partial
from pathlib import Path
import itertools

import torch

import pycocotools
import detectron2
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.structures import BoxMode
from detectron2.data import datasets, DatasetCatalog, MetadataCatalog, build_detection_train_loader, build_detection_test_loader
from detectron2.data import transforms as T
from detectron2.data import detection_utils as utils
from detectron2.evaluation import COCOEvaluator, verify_results
from detectron2.modeling import GeneralizedRCNNWithTTA
from detectron2.data.transforms import TransformGen
from detectron2.utils.logger import setup_logger
setup_logger()

from fvcore.transforms.transform import TransformList, Transform, NoOpTransform
from contextlib import contextmanager

import torch.nn as nn
from collections import Counter

In [ ]:
data_dir = Path('../input/sartorius-cell-instance-segmentation')
train_img_dir = Path(data_dir / 'train')
test_img_dir = Path(data_dir / 'test')

sub_path = Path(data_dir / 'sample_submission.csv')

In [ ]:
df = pd.read_csv(data_dir / 'train.csv')
sub_df = pd.read_csv(sub_path)

# Some EDA

In [ ]:
df.head()

# How many cell type?

In [ ]:
# List of unique cell types:
print(list(set(df['cell_type'].tolist())))

### We have 3 differents types of cells, We will use them on MetadataCatalog.get(dataset_label).thing_classes
1. **astro**
2. **cort**
3. **shsy5y**

## Number of instances

In [ ]:
# Number of instances
df.shape[0]

## Number of unique instances

In [ ]:
print('There are '+str(len(list(set(df['id'].tolist()))))+' unique image')

In [ ]:
# Groupby image_id to get an image per row
unique_images = df.groupby('id')[['annotation', 'width', 'height', 'cell_type']].agg(lambda x: list(x)).reset_index()
unique_images.head()

## Number of segmentations per image

In [ ]:
cnt = {}

for idx, row in unique_images.iterrows(): 
    length = len(unique_images.annotation.iloc[idx])
    image_id = unique_images.id.iloc[idx]
    cnt.update({id: length})

df_count_segments = pd.Series(data=cnt)

In [ ]:
print(f'Average number of segmentations per image: {df_count_segments.values.mean()}')
print(f'Max number of segmentations: {df_count_segments.values.max()}')
print(f'Min number of segmentations: {df_count_segments.values.min()}')

# Let's take a look to some images 

In [ ]:
# ref: https://www.kaggle.com/inversion/run-length-decoding-quick-start
# ref: updated by https://www.kaggle.com/ihelon/cell-segmentation-run-length-decoding
def rle_decode(mask_rle, shape, color=1):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height, width, channels) of array to return 
    color: color for the mask
    Returns numpy array (mask)

    '''
    s = mask_rle.split()
    
    starts = list(map(lambda x: int(x) - 1, s[0::2]))
    lengths = list(map(int, s[1::2]))
    ends = [x + y for x, y in zip(starts, lengths)]
    
    img = np.zeros((shape[0] * shape[1], shape[2]), dtype=np.float32)
            
    for start, end in zip(starts, ends):
        img[start : end] = color
    
    return img.reshape(shape)

In [ ]:
def plot_masks(image_id, colors=True):
    labels = df[df["id"] == image_id]["annotation"].tolist()

    if colors:
        mask = np.zeros((520, 704, 3))
        for label in labels:
            mask += rle_decode(label, shape=(520, 704, 3), color=np.random.rand(3))
    else:
        mask = np.zeros((520, 704, 1))
        for label in labels:
            mask += rle_decode(label, shape=(520, 704, 1))
    mask = mask.clip(0, 1)

    image = cv2.imread(f"../input/sartorius-cell-instance-segmentation/train/{image_id}.png")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(16, 32))
    plt.subplot(3, 1, 1)
    plt.imshow(image)
    plt.axis("off")
    plt.subplot(3, 1, 2)
    plt.imshow(image)
    plt.imshow(mask, alpha=0.5)
    plt.axis("off")
    plt.subplot(3, 1, 3)
    plt.imshow(mask)
    plt.axis("off")
    
    plt.show();

In [ ]:
plot_masks("ffdb3cc02eef", colors=False)

In [ ]:
plot_masks("ffdb3cc02eef", colors=True)

In [ ]:
plot_masks("73df2962444f", colors=True)

In [ ]:
plot_masks("13325f865bb0", colors=True)

# Train Detectron2 on a custom dataset

In [ ]:
cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model

cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)

im = cv2.imread('../input/sartorius-cell-instance-segmentation/train/0030fd0e6378.png')
outputs = predictor(im[..., ::-1])

# Create Detectron2 dataset dict

In [ ]:
def get_cell_dict(df, img_folder):
    
    grps = df['id'].unique().tolist()
    df_group = df.groupby('id')
    dataset_dicts = []
    
    for idx, image_name in enumerate(tqdm(grps)):
        
        # Get all instances of an image
        group = df_group.get_group(image_name)
        
        record = defaultdict()
        
        # Full image path 
        file_path = os.path.join(img_folder, image_name + '.jpg')
        
        record['height'] = int(520)
        record['width'] = int(704)
        record['file_name'] = file_path
        record['image_id'] = idx
        
        objs = []
        
        # Iterate over the group's rows as namedtuples
        for ant in group.itertuples():

            
            s = ant.annotation[1:-1]
            #print(s)
            s = s.split()
    
            starts = list(map(lambda x: int(x) - 1, s[0::2]))
            lengths = list(map(int, s[1::2]))
            
            
            
            xmin, ymin, xmax, ymax = min(starts), min(lengths), max(starts), max(lengths)
            
            poly = []
            for x,y in zip(starts, lengths):
                poly.append((x,y))
            
            
            poly = list(itertools.chain.from_iterable(poly))
            
            obj = {
                'bbox': [xmin, ymin, xmax, ymax], # change to XYXY format. Original was in XYWH
                'bbox_mode': BoxMode.XYXY_ABS,
                'segmentation': [poly],
                'category_id': ant.cell_type[0], # only 1 category for this dataset
                'iscrowd': 0
                  }
            
            objs.append(obj)
        record['annotations'] = objs
        dataset_dicts.append(record)

    return dataset_dicts

##saaa7

In [ ]:
class CutOut(Transform):
    
    def __init__(self, box_size=50, prob_cutmix=0.8):
        super().__init__()
        
        self.box_size = box_size
        self.prob_cutmix = prob_cutmix
        
    def apply_image(self, img):
        
        if random.random() > self.prob_cutmix:
            
            h, w = img.shape[:2]
            num_rand = np.random.randint(10, 20)
            for num_cut in range(num_rand):
                x_rand, y_rand = random.randint(0, w-self.box_size), random.randint(0, h-self.box_size)
                img[x_rand:x_rand+self.box_size, y_rand:y_rand+self.box_size, :] = 0
        
        return np.asarray(img)
    
    def apply_coords(self, coords):
        return coords.astype(np.float32)

In [ ]:
class DatasetMapper:
    """
    A callable which takes a dataset dict in Detectron2 Dataset format,
    and map it into a format used by the model.

    This is a custom version of the DatasetMapper. The only different with Detectron2's 
    DatasetMapper is that we extract attributes from our dataset_dict. 
    """

    def __init__(self, cfg, is_train=True):
        if cfg.INPUT.CROP.ENABLED and is_train:
            self.crop_gen = T.RandomCrop(cfg.INPUT.CROP.TYPE, cfg.INPUT.CROP.SIZE)
            logging.getLogger(__name__).info("CropGen used in training: " + str(self.crop_gen))
        else:
            self.crop_gen = None
        
        self.tfm_gens = [T.RandomBrightness(0.8, 1.8),
                         T.RandomContrast(0.6, 1.3),
                         T.RandomSaturation(0.8, 1.4),
                         T.RandomRotation(angle=[90, 90]),
                         T.RandomLighting(0.7),
                         T.RandomFlip(prob=0.4, horizontal=False, vertical=True),
                         T.RandomCrop('relative_range', (0.4, 0.6)),
                         CutOut()
                        ]

        # self.tfm_gens = utils.build_transform_gen(cfg, is_train)

        # fmt: off
        self.img_format     = cfg.INPUT.FORMAT
        self.mask_on        = cfg.MODEL.MASK_ON
        self.mask_format    = cfg.INPUT.MASK_FORMAT
        self.keypoint_on    = cfg.MODEL.KEYPOINT_ON
        self.load_proposals = cfg.MODEL.LOAD_PROPOSALS
        # fmt: on
        if self.keypoint_on and is_train:
            # Flip only makes sense in training
            self.keypoint_hflip_indices = utils.create_keypoint_hflip_indices(cfg.DATASETS.TRAIN)
        else:
            self.keypoint_hflip_indices = None

        if self.load_proposals:
            self.min_box_side_len = cfg.MODEL.PROPOSAL_GENERATOR.MIN_SIZE
            self.proposal_topk = (
                cfg.DATASETS.PRECOMPUTED_PROPOSAL_TOPK_TRAIN
                if is_train
                else cfg.DATASETS.PRECOMPUTED_PROPOSAL_TOPK_TEST
            )
        self.is_train = is_train

    def __call__(self, dataset_dict):
        """
        Args:
            dataset_dict (dict): Metadata of one image, in Detectron2 Dataset format.

        Returns:
            dict: a format that builtin models in detectron2 accept
        """
        dataset_dict = copy.deepcopy(dataset_dict)  # it will be modified by code below
        # USER: Write your own image loading if it's not from a file
        image = utils.read_image(dataset_dict["file_name"], format=self.img_format)
        utils.check_image_size(dataset_dict, image)

        if "annotations" not in dataset_dict:
            image, transforms = T.apply_transform_gens(
                ([self.crop_gen] if self.crop_gen else []) + self.tfm_gens, image
            )
        else:
            # Crop around an instance if there are instances in the image.
            # USER: Remove if you don't use cropping
            if self.crop_gen:
                crop_tfm = utils.gen_crop_transform_with_instance(
                    self.crop_gen.get_crop_size(image.shape[:2]),
                    image.shape[:2],
                    np.random.choice(dataset_dict["annotations"]),
                )
                image = crop_tfm.apply_image(image)
            image, transforms = T.apply_transform_gens(self.tfm_gens, image)
            if self.crop_gen:
                transforms = crop_tfm + transforms

        image_shape = image.shape[:2]  # h, w

        # Pytorch's dataloader is efficient on torch.Tensor due to shared-memory,
        # but not efficient on large generic data structures due to the use of pickle & mp.Queue.
        # Therefore it's important to use torch.Tensor.
        dataset_dict["image"] = torch.as_tensor(np.ascontiguousarray(image.transpose(2, 0, 1)))

        # USER: Remove if you don't use pre-computed proposals.
        if self.load_proposals:
            utils.transform_proposals(
                dataset_dict, image_shape, transforms, self.min_box_side_len, self.proposal_topk
            )

        if not self.is_train:
            # USER: Modify this if you want to keep them for some reason.
            dataset_dict.pop("annotations", None)
            dataset_dict.pop("sem_seg_file_name", None)
            return dataset_dict

        if "annotations" in dataset_dict:
            # USER: Modify this if you want to keep them for some reason.
            for anno in dataset_dict["annotations"]:
                if not self.mask_on:
                    anno.pop("segmentation", None)
                if not self.keypoint_on:
                    anno.pop("keypoints", None)

            # USER: Implement additional transformations if you have other types of data
            annos = [
                utils.transform_instance_annotations(
                    obj, transforms, image_shape, keypoint_hflip_indices=self.keypoint_hflip_indices
                )
                for obj in dataset_dict.pop("annotations")
                if obj.get("iscrowd", 0) == 0
            ]
            instances = utils.annotations_to_instances(
                annos, image_shape, mask_format=self.mask_format
            )
            # Create a tight bounding box from masks, useful when image is cropped
            if self.crop_gen and instances.has("gt_masks"):
                instances.gt_boxes = instances.gt_masks.get_bounding_boxes()           
                          
            dataset_dict["instances"] = utils.filter_empty_instances(instances)
            
             # USER: Remove if you don't do semantic/panoptic segmentation.
        if "sem_seg_file_name" in dataset_dict:
            with PathManager.open(dataset_dict.pop("sem_seg_file_name"), "rb") as f:
                sem_seg_gt = Image.open(f)
                sem_seg_gt = np.asarray(sem_seg_gt, dtype="uint8")
            sem_seg_gt = transforms.apply_segmentation(sem_seg_gt)
            sem_seg_gt = torch.as_tensor(sem_seg_gt.astype("long"))
            dataset_dict["sem_seg"] = sem_seg_gt

        return dataset_dict

In [ ]:
class CellTrainer(DefaultTrainer):
    
    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=DatasetMapper(cfg))
    
    @classmethod
    def build_test_loader(cls, cfg, dataset_name):
        return build_detection_test_loader(cfg, dataset_name, mapper=DatasetMapper(cfg))

In [ ]:
def train_test_datasets(df, n_sample_size=-1, train_test_split=0.8):
    
    # Option to set a smaller df size for testing
    if n_sample_size == -1:
        n_sample_size = df.shape[0]
    elif n_sample_size != -1:
        n_sample_size = df[:n_sample_size].shape[0]

    # Split df into train / test dataframes
    n_train = round(n_sample_size * train_test_split)
    n_test = n_sample_size - n_train

    df_train = df[:n_train].copy()
    df_test = df[-n_test:].copy()
    
    return df_train, df_test

In [ ]:
def register_dataset(df, dataset_label='cell_train', image_dir=train_img_dir):
    
    # Register dataset - if dataset is already registered, give it a new name    
    try:
        DatasetCatalog.register(dataset_label, lambda d=df: get_cell_dict(df, image_dir))
        MetadataCatalog.get(dataset_label).thing_classes = ['astro', 'cort', 'shsy5y']
    except:
        # Add random int to dataset name to not run into 'Already registered' error
        n = random.randint(1, 1000)
        dataset_label = dataset_label + str(n)
        DatasetCatalog.register(dataset_label, lambda d=df: get_cell_dict(df, image_dir))
        MetadataCatalog.get(dataset_label).thing_classes = ['astro', 'cort', 'shsy5y']
        
    return MetadataCatalog.get(dataset_label), dataset_label

In [ ]:
# Register train dataset
metadata, train_dataset = register_dataset(df)

In [ ]:
# Register test dataset
metadata, test_dataset = register_dataset(sub_df, dataset_label='cell_test', image_dir=test_img_dir)

In [ ]:
cell_dict = get_cell_dict(df, train_img_dir)

In [ ]:
MODEL_USE = 'faster_rcnn'
if MODEL_USE == 'faster_rcnn':
    MODEL_PATH = 'COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml'
    WEIGHT_PATH = '../input/r101fpn400/model_final_f96b26.pkl'
elif MODEL_USE == 'retinanet':
    MODEL_PATH = 'COCO-Detection/retinanet_R_101_FPN_3x.yaml'
    WEIGHT_PATH = '/kaggle/input/model-final/model_retina.pth' 
elif MODEL_USE == 'mask_rcnn':
    MODEL_PATH = 'COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml'
    WEIGHT_PATH = 'COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml'
elif MODEL_USE == 'cascade_mask_rcnn':
    MODEL_PATH = 'Misc/cascade_mask_rcnn_R_50_FPN_3x.yaml'
    WEIGHT_PATH = '/kaggle/input/model-cascade-10000/model_final_cascade_10000.pth'

def cfg_setup():
    
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file(MODEL_PATH))
    cfg.MODEL.WEIGHTS = WEIGHT_PATH # model_zoo.get_checkpoint_url(WEIGHT_PATH)  
    cfg.MODEL.RETINANET.NUM_CLASSES = 3
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3

    cfg.DATASETS.TRAIN = (train_dataset,)
    cfg.DATASETS.TEST = ()
    cfg.DATALOADER.NUM_WORKERS = 4

    cfg.SOLVER.IMS_PER_BATCH = 4
    cfg.SOLVER.LR_SCHEDULER_NAME = 'WarmupCosineLR'
    cfg.SOLVER.BASE_LS = 0.0002
    cfg.SOLVER.MAX_ITER = 500
    os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
        
    return cfg

In [ ]:
cfg = cfg_setup()
#trainer = CellTrainer(cfg)   

# I am still bilding this notebook, an uptove will motivate me too much, Thanks n_n!
reference: [Link](https://www.kaggle.com/julienbeaulieu/detectron2-wheat-detection-eda-training-eval)